# Training the model

In [15]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [16]:
# Define the transformation for data augmentation
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [17]:
data_dir = 'dataset/'
image_dataset = datasets.ImageFolder(root=data_dir,transform=data_transforms)
data_loaders = DataLoader(image_dataset, batch_size=32, shuffle=True, num_workers=4)

In [18]:
dataset_size = len(image_dataset)
class_names =  image_dataset.classes

In [19]:
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes in your dataset
num_classes = len(class_names)  # Number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [21]:
from sklearn.model_selection import train_test_split

# Split your dataset into training (70%) and temporary (30%)
train_data, temp_data = train_test_split(image_dataset, test_size=0.3, random_state=42)

# Split the temporary data into validation (50%) and test (50%)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [22]:
batch_size = 32  # You can adjust the batch size

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_data, batch_size=batch_size, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, num_workers=4)

In [23]:
num_epochs = 10  # You can adjust the number of training epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        # ... your training code ...
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs} Training Loss: {running_loss / len(train_data):.4f}')

    # Validation Step
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            # ... your validation code ...
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}/{num_epochs} Validation Accuracy: {accuracy:.2f}%')

print('Training complete')


Epoch 1/10 Training Loss: 0.0549
Epoch 1/10 Validation Accuracy: 21.05%
Epoch 2/10 Training Loss: 0.0476
Epoch 2/10 Validation Accuracy: 26.32%
Epoch 3/10 Training Loss: 0.0375
Epoch 3/10 Validation Accuracy: 42.11%
Epoch 4/10 Training Loss: 0.0301
Epoch 4/10 Validation Accuracy: 57.89%
Epoch 5/10 Training Loss: 0.0227
Epoch 5/10 Validation Accuracy: 78.95%
Epoch 6/10 Training Loss: 0.0169
Epoch 6/10 Validation Accuracy: 89.47%
Epoch 7/10 Training Loss: 0.0131
Epoch 7/10 Validation Accuracy: 84.21%
Epoch 8/10 Training Loss: 0.0106
Epoch 8/10 Validation Accuracy: 84.21%
Epoch 9/10 Training Loss: 0.0079
Epoch 9/10 Validation Accuracy: 84.21%
Epoch 10/10 Training Loss: 0.0067
Epoch 10/10 Validation Accuracy: 89.47%
Training complete


In [24]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        # ... your testing code ...
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


Accuracy on the test set: 94.74%


# Predicting a single image

In [25]:
from PIL import Image
import torchvision.transforms as transforms

# Load the image
image_path = 'image_to_predict.jpg'  # Replace with the path to your image
image = Image.open(image_path)

# Define the transformation
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply the transformation to the image
input_tensor = data_transforms(image)
input_tensor = input_tensor.unsqueeze(0)  # Add a batch dimension

In [26]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    outputs = model(input_tensor)

# Get the predicted class
_, predicted = torch.max(outputs, 1)

# Convert the predicted class to a human-readable label
predicted_class = class_names[predicted.item()]

In [27]:
print(f'Predicted class: {predicted_class}')

Predicted class: 2
